In [ ]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo

In [ ]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
# Define database and collection
db = client.craigslist_db
collection = db.items

In [ ]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [ ]:
results = soup.find_all('li', class_='slide')

# Loop through returned results
for result in results:
    # Error handling
    try:
        # Identify and return title of listing
        mars_title = result.find('div', class_='content title').text
        # Identify and return price of listing
        mars_p = result.div['article_teaser_body'].text
        # Identify and return link to listing
        link = result.a['href']

        # Run only if title, price, and link are available
        if (mars_title and mars_p and link):
            # Print results
            print('-------------')
            print(mars_title)
            print(mars_p)
            print(link)

            # Dictionary to be inserted as a MongoDB document
            post = {
                'title': mars_title,
                'price': mars_p,
                'url': link
            }

            collection.insert_one(post)

    except Exception as e:
        print(e)